In [1]:
# %load_ext autoreload
# %autoreload 2

import os, json
import nibabel as nb

from keras import metrics 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LocallyConnected1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.utils.vis_utils import plot_model

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import hrf_estimation
import scipy.signal as signal
import scipy.stats as stats

from joblib import Parallel, delayed

%matplotlib inline

Using TensorFlow backend.
/home/knapen/software/miniconda3/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:908: MatplotlibDeprecationWarning: The backend.qt4 rcParam was deprecated in version 2.2.  In order to force the use of a specific Qt binding, either import that binding first, or set the QT_API environment variable.
  mplDeprecation)
/home/knapen/software/miniconda3/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:908: MatplotlibDeprecationWarning: The backend.qt4 rcParam was deprecated in version 2.2.  In order to force the use of a specific Qt binding, either import that binding first, or set the QT_API environment variable.
  mplDeprecation)


In [3]:
# requires pfpy be installed - preferably with python setup.py develop
from prfpy.rf import *
from prfpy.timecourse import *
from prfpy.stimulus import PRFStimulus2D
from prfpy.grid import Iso2DGaussianGridder
from prfpy.fit import Iso2DGaussianFitter

ModuleNotFoundError: No module named 'nilearn'

In [ ]:
stim_dm = np.load('../data/dm.npy').T # transpose for final dimension time
prf_stim = PRFStimulus2D(screen_size_cm=70, 
                         screen_distance_cm=125, 
                         design_matrix=stim_dm, 
                         TR=1.6)

grid_nr = 12
max_ecc_size = 20
sizes, eccs, polars = max_ecc_size * np.linspace(0.25,1,grid_nr)**2, \
                    max_ecc_size * np.linspace(0.1,1,grid_nr)**2, \
                        np.linspace(0, 2*np.pi, grid_nr)

gg = Iso2DGaussianGridder(stimulus=prf_stim)

simulated_data = np.array([gg.return_single_prediction(mu_x=1.23,
                                            mu_y=-2.4,
                                            size=1.12, 
                                            n=1.0,
                                            beta=3.21,
                                            baseline=-0.76),
                  gg.return_single_prediction(mu_x=1.23,
                                            mu_y=-2.4,
                                            size=2.12, 
                                            n=1.0,
                                            beta=3.21,
                                            baseline=-0.76)]).squeeze()

gf = Iso2DGaussianFitter(data=simulated_data, gridder=gg, n_jobs=2, fit_css=False)


In [ ]:
gf.grid_fit(ecc_grid=eccs,
                 polar_grid=polars,
                 size_grid=sizes)

In [ ]:
# print(gf.gridsearch_params.shape)
gf.iterative_fit(rsq_threshold=0)
print(gf.iterative_search_params)

In [ ]:
f, ss = plt.subplots(1,2, figsize=(16,5)) 
ss[0].plot(simulated_data.T)
ss[1].plot(np.array([gg.return_single_prediction(*ps[:-1]) for ps in gf.iterative_search_params]).squeeze().T)

In [ ]:
np.unique(gf.gridder.ns)